In [1]:
import pandas as pd
import numpy as np
from CustomImputers import *

In [2]:
df=pd.read_csv('train_cleaned_outcome_imputed.csv')

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

predictors = df.columns.tolist()
if 'id' in predictors:
    predictors.remove('id')
if 'sii' in predictors:
    predictors.remove('sii')
predictors = [x for x in predictors if 'PCIAT' not in x]
predictors = [x for x in predictors if 'Season' not in x]

pipe_mice = Pipeline([('mice_impute', Custom_MICE_Imputer()),
                    ('add_zones', FunctionTransformer(zone_encoder)),
                    ('lr', LinearRegression())])

pipe_mice.fit(df[predictors],df['PCIAT-PCIAT_Total'])

df_pred_mice = pipe_mice.predict(df[predictors])


In [4]:
df_pred_mice


array([13.73019129, 38.93127126, 21.36338431, ..., 31.32001239,
       47.17682469, 14.17289674])

In [5]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df,
                                                       shuffle = True,
                                                       random_state = 440,
                                                       test_size = .2)


In [6]:
df_train.head()

df_train.reset_index(inplace=True)

In [7]:
pipe_mice.fit(df_train[predictors],df_train['PCIAT-PCIAT_Total'])

df_train_mice = pipe_mice.predict(df_train[predictors])

In [8]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5,
              shuffle = True,
              random_state = 582)
df_kfold=kfold.split(df)


In [ ]:
from sklearn.metrics import root_mean_squared_error

In [15]:
rmses = np.zeros((2, 5))

slr = LinearRegression()
mlr = LinearRegression()

i=0
for train_index, test_index in kfold.split(df):
    ## get the kfold training data
    train = df.iloc[train_index,:]
    train.reset_index(inplace=True)
    
    ## get the holdout data
    holdout = df.iloc[test_index,:]
    holdout.reset_index(inplace=True)

    ## Fit both models
    slr.fit(train[['Basic_Demos-Age']], train[['sii']])
    mlr.fit(train[predictors], train[['sii']])

    ## Use both models to generate predictions on the holdout set
    slr_preds = slr.predict(holdout[['Basic_Demos-Age']])
    mlr_preds = mlr.predict(holdout[predictors])

    ## Record the rmses
    rmses[0,i] = root_mean_squared_error(y_holdout, slr_preds)
    rmses[1,i] = root_mean_squared_error(y_holdout, mlr_preds)
    i+=1

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [13]:
predictors

['Basic_Demos-Age',
 'Basic_Demos-Sex',
 'CGAS-CGAS_Score',
 'Physical-BMI',
 'Physical-Height',
 'Physical-Weight',
 'Physical-Waist_Circumference',
 'Physical-Diastolic_BP',
 'Physical-HeartRate',
 'Physical-Systolic_BP',
 'Fitness_Endurance-Max_Stage',
 'FGC-FGC_CU',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_PU',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_TL',
 'FGC-FGC_TL_Zone',
 'BIA-BIA_Activity_Level_num',
 'BIA-BIA_FFM',
 'BIA-BIA_FFMI',
 'BIA-BIA_FMI',
 'BIA-BIA_Fat',
 'BIA-BIA_Frame_num',
 'SDS-SDS_Total_Raw',
 'PreInt_EduHx-computerinternet_hoursday',
 'ENMO_Avg_Active_Days_MVPA192',
 'ENMO_Avg_Active_Days_MVPA110',
 'Positive_Anglez_Active_Days',
 'FGC-FGC_SR',
 'FGC-FGC_SR_Zone',
 'PAQ_Total',
 'PAQ_Zone',
 'Fitness_Endurance_Total_Time_Sec']